In [ ]:
# Playlist API Call

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import time

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

# Top playlist:
# 37i9dQZF1DX0XUsuxWHRQd
# Trackstarz playlist
# 2djOtO1aa0Fg0NzFFuU6aO

# playlist_uri = 'spotify:playlist:37i9dQZF1DX0XUsuxWHRQd'

def getPlaylistResults(playlistURI, jsonFileName):
    playlistResults = spotify.playlist(playlist_id=playlistURI)

    playlistDict = {"track_uri": [], "artist_uri": [], "album_uri": [], "artist_name": [], "album_name": [], "release_date": [], "track_name": [], "duration_ms": [], "explicit": [], "popularity": []}

    for idx, item in enumerate(playlistResults['tracks']['items']):
        track = item['track']
        playlistDict['track_uri'].append(track['uri'])
        playlistDict['artist_uri'].append(track['artists'][0]['uri'])
        playlistDict['album_uri'].append(track['album']['uri'])
        playlistDict['artist_name'].append(track['artists'][0]['name'])
        playlistDict['album_name'].append(track['album']['name'])
        playlistDict['release_date'].append(track['album']['release_date'])
        playlistDict['track_name'].append(track['name'])
        playlistDict['duration_ms'].append(track['duration_ms'])
        playlistDict['explicit'].append(track['explicit'])
        playlistDict['popularity'].append(track['popularity'])

    with open(jsonFileName, 'w') as outfile:
        json.dump(playlistDict, outfile)

getPlaylistResults('spotify:playlist:37i9dQZF1DX0XUsuxWHRQd', 'HHPlaylistDict.json')

time.sleep(1)

getPlaylistResults('spotify:playlist:2djOtO1aa0Fg0NzFFuU6aO', 'CHHPlaylistDict.json')

# PlaylistDF = pd.DataFrame(playlistDict, index=playlistIndex)
# PlaylistDF





In [ ]:
# Artist API Call

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import time

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))


with open('HHPlaylistDict.json', 'r') as openfile:
    HHPlaylistDict = json.load(openfile)

with open('CHHPlaylistDict.json', 'r') as openfile:
    CHHPlaylistDict = json.load(openfile)


def appendArtistsToDict(artistResults, artistDict):

    for idx, item in enumerate(artistResults['artists']):
        artistDict['artist_uri'].append(item['uri'])
        artistDict['artist_name'].append(item['name'])
        artistDict['popularity'].append(item['popularity'])
        artistDict['followers'].append(item['followers']['total'])


def getArtistsFromPlaylist(playlistDict, jsonFileName):
    artistList = []

    for item in playlistDict['artist_uri']:
        if item in artistList:
            artistList = artistList
        else:
            artistList.append(item)

    artistListDict = {'Under50': [], 'Above50': []}
    for value in artistList:
        if artistList.index(value) < 49:
            artistListDict['Under50'].append(value)
        else:
            artistListDict['Above50'].append(value)

    # print(len(artistListDict['Above50']))
    
    resultsDict = {'Under50': [], 'Above50': []}
    artistDict = {'artist_uri': [], 'artist_name': [], 'popularity': [], 'followers': []}

    resultsDict['Under50'].append(spotify.artists(artistListDict['Under50']))
    appendArtistsToDict(resultsDict['Under50'][0], artistDict)

    if len(artistListDict['Above50']) > 0:
        time.sleep(1)
        resultsDict['Above50'].append(spotify.artists(artistListDict['Above50']))
        appendArtistsToDict(resultsDict['Above50'][0], artistDict)

    with open(jsonFileName, 'w') as outfile:
        json.dump(artistDict, outfile)


    

getArtistsFromPlaylist(HHPlaylistDict, 'HHArtistDict.json')
time.sleep(1)
getArtistsFromPlaylist(CHHPlaylistDict, 'CHHArtistDict.json')

In [ ]:
# Read data from Excel/API

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import time

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

playlistDict = {}

# with open('PlaylistDict.json', 'r') as openfile:
#     playlistDict = json.load(openfile)

# PlaylistDF = pd.DataFrame(playlistDict)

HHPlaylistDF = pd.read_excel('PlaylistAnalysis.xlsx',sheet_name='HHPlaylist')
# CHHPlaylistDF = pd.read_excel('PlaylistAnalysis.xlsx',sheet_name='CHHPlaylist')



In [ ]:
# Process Album Data

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import time

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

playlistDict = {}

with open('HHPlaylistDict.json', 'r') as openfile:
    HHPlaylistDict = json.load(openfile)

with open('CHHPlaylistDict.json', 'r') as openfile:
    CHHPlaylistDict = json.load(openfile)

def getAlbumAPIResults(albumListDict):
    albumResults = {'albumResult0_19': [], 'albumResult20_39': [], 'albumResult40_69': []}
    
    albumResults['albumResult0_19'].append(spotify.albums(albumListDict['albumList0_19']))
    time.sleep(1)
    albumResults['albumResult20_39'].append(spotify.albums(albumListDict['albumList20_39']))
    time.sleep(1)
    albumResults['albumResult40_69'].append(spotify.albums(albumListDict['albumList40_69']))
    
    return albumResults

# Create a function for each list of albums:
def addAlbumsToDict(APIResult, containerDict):
    for idx, item in enumerate(APIResult['albums']):
        containerDict['album_uri'].append(item['uri'])
        containerDict['artist_uri'].append(item['artists'][0]['uri'])
        containerDict['album_name'].append(item['name'])
        containerDict['artist_name'].append(item['artists'][0]['name'])
        containerDict['popularity'].append(item['popularity'])
        containerDict['label'].append(item['label'])
        containerDict['release_date'].append(item['release_date'])
        containerDict['total_tracks'].append(item['total_tracks'])

def createAlbumFile(playlist, fileName):
    albumList = []
    for item in playlist['album_uri']:
        if item in albumList:
            albumList = albumList
        else:
            albumList.append(item)

    albumListDict = {'albumList0_19': [], 'albumList20_39': [], 'albumList40_69': []}

    for value in albumList:
        if albumList.index(value) <= 19:
            albumListDict['albumList0_19'].append(value)
        elif albumList.index(value) <= 39:
            albumListDict['albumList20_39'].append(value)
        elif albumList.index(value) <= 69:
            albumListDict['albumList40_69'].append(value)

    albumResults = getAlbumAPIResults(albumListDict)

    albumDict = {'album_uri': [], 'album_name': [],'artist_uri': [], 'artist_name': [], 'popularity': [], 'label': [], 'release_date': [], 'total_tracks': []}

    addAlbumsToDict(list(albumResults.values())[0][0], albumDict)
    addAlbumsToDict(list(albumResults.values())[1][0], albumDict)
    addAlbumsToDict(list(albumResults.values())[2][0], albumDict)
    
    with open(fileName, 'w') as outfile:
        json.dump(albumDict, outfile)

createAlbumFile(HHPlaylistDict, 'HHAlbumDict.json')
time.sleep(1)
createAlbumFile(CHHPlaylistDict, 'CHHAlbumDict.json')

# list(HHalbumListDict.values())[0][0]





In [ ]:
# Get Track Results

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import time

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

with open('HHPlaylistDict.json', 'r') as openfile:
    HHPlaylistDict = json.load(openfile)

with open('CHHPlaylistDict.json', 'r') as openfile:
    CHHPlaylistDict = json.load(openfile)


def getTrackAPIResults(trackListDict):
    trackResults = {'trackResult0_19': [], 'trackResult20_39': [], 'trackResult40_69': []}
    trackResults['trackResult0_19'].append(spotify.audio_features(trackListDict['trackList0_19']))
    time.sleep(3)
    trackResults['trackResult20_39'].append(spotify.audio_features(trackListDict['trackList20_39']))
    time.sleep(3)
    trackResults['trackResult40_69'].append(spotify.audio_features(trackListDict['trackList40_69']))
    return trackResults

def addTracksToDict(APIResult, containerDict):
    for idx, item in enumerate(APIResult):
        containerDict['uri'].append(item['uri'])
        containerDict['duration_ms'].append(item['duration_ms'])
        containerDict['time_signature'].append(item['time_signature'])
        containerDict['tempo'].append(item['tempo'])
        containerDict['valence'].append(item['valence'])
        containerDict['liveness'].append(item['liveness'])
        containerDict['instrumentalness'].append(item['instrumentalness'])
        containerDict['acousticness'].append(item['acousticness'])
        containerDict['speechiness'].append(item['speechiness'])
        containerDict['mode'].append(item['mode'])
        containerDict['loudness'].append(item['loudness'])
        containerDict['key'].append(item['key'])
        containerDict['energy'].append(item['energy'])
        containerDict['danceability'].append(item['danceability'])

def createTrackFile(playlist, fileName):
    trackList = []
    for item in playlist['track_uri']:
        if item in trackList:
            trackList = trackList
        else:
            trackList.append(item)

    trackListDict = {'trackList0_19': [], 'trackList20_39': [], 'trackList40_69': []}

    for value in trackList:
        if trackList.index(value) <= 19:
            trackListDict['trackList0_19'].append(value)
        elif trackList.index(value) <= 39:
            trackListDict['trackList20_39'].append(value)
        elif trackList.index(value) <= 69:
            trackListDict['trackList40_69'].append(value)

    trackResults = getTrackAPIResults(trackListDict)
    
    trackDict = {'uri': [], 'duration_ms': [],'time_signature': [], 'tempo': [], 'valence': [], 'liveness': [], 'instrumentalness': [], 'acousticness': [], 'speechiness': [], 'mode': [], 'loudness': [], 'key': [], 'energy': [], 'danceability': []}

    addTracksToDict(list(trackResults.values())[0][0], trackDict)
    addTracksToDict(list(trackResults.values())[1][0], trackDict)
    addTracksToDict(list(trackResults.values())[2][0], trackDict)

    with open(fileName, 'w') as outfile:
        json.dump(trackDict, outfile)

createTrackFile(HHPlaylistDict, 'HHTrackFeaturesDict.json')
createTrackFile(CHHPlaylistDict, 'CHHTrackFeaturesDict.json')

In [16]:
# Write Dataframes to Excel file

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import datetime as datetime

def openFileToDataFrame(fileName):
    with open(fileName, 'r') as openfile:
        file = json.load(openfile)

    df = pd.DataFrame(file)
    return df

xlxWriter = pd.ExcelWriter('SpotifyPlaylistData.xlsx', engine='xlsxwriter')

openFileToDataFrame('HHPlaylistDict.json').to_excel(xlxWriter, sheet_name='HHPlaylist')
openFileToDataFrame('CHHPlaylistDict.json').to_excel(xlxWriter, sheet_name='CHHPlaylist')
openFileToDataFrame('HHArtistDict.json').to_excel(xlxWriter, sheet_name='HHArtists')
openFileToDataFrame('CHHArtistDict.json').to_excel(xlxWriter, sheet_name='CHHArtists')
openFileToDataFrame('HHAlbumDict.json').to_excel(xlxWriter, sheet_name='HHAlbums')
openFileToDataFrame('CHHAlbumDict.json').to_excel(xlxWriter, sheet_name='CHHAlbums')
openFileToDataFrame('HHTrackFeaturesDict.json').to_excel(xlxWriter, sheet_name='HHTrackFeatures')
openFileToDataFrame('CHHTrackFeaturesDict.json').to_excel(xlxWriter, sheet_name='CHHTrackFeatures')


xlxWriter.close()